In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [63]:
headers = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
page = requests.get("https://www.profital.ch/de/fl/81450-migros-filialen?page=1", headers=headers)

In [64]:
print(page.status_code)

200


In [65]:
soup = BeautifulSoup(page.content, "html.parser") # converts the page content into a beautifulsoup object

In [76]:
stores = soup.findAll("div", {"class": "wrapper-store-top"})

In [77]:
len(stores)

15

In [ ]:
<a href="/de/zuerich-shopvillemaert" class="sc-hWRnEO jtIteb"><aside aria-hidden="true" data-test-id="storefinder_stores-list_item-logo-0150140" class="sc-kKXBxx kpuDlJ"><img src="//image.migros.ch/filialen/200x200/b7ed598472b2aa065658425a494316272885c3ff/m-de.png" alt="M-Migros" class="sc-hTZtdg dKwuNP"></aside><div><div><span data-test-id="storefinder_stores-list_item-name-0150140" class="sc-fIYMun kWWXCI">Zürich - ShopVilleMärt</span><div aria-label="129 m Distanz in Luftlinie zwischen Filiale und geografischem Mittelpunkt von &quot;&quot;" title="Distanz in Luftlinie zwischen Filiale und geografischem Mittelpunkt von &quot;&quot;" class="sc-gATIKf ckjCyl"><span aria-hidden="true"><img src="https://cdn.migros.ch/ch.migros.msrc/img/stores/20200317/map-markers/migros/search-location.svg" class="sc-hxqDqe eYaJeZ">129 m</span></div></div><p data-test-id="storefinder_stores-list_item-address-0150140" class="sc-cSa-DZW dkKZrh">Shop Ville, 8001 Zürich</p><div class="sc-kNPvZf fXSnaN">Heute bis 21:00 geöffnet</div></div></a>

In [91]:
stores[0].find("address").text.strip().split("\n")[0].strip()

'Wengistrasse 7'

In [90]:
stores[0].find("span", {"class": "title-store"}).text

'Migros'

In [93]:

def get_filialen():
    headers = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    cols = [ "name", "address"]
    df = pd.DataFrame(columns = cols)
    for j in range(1,44):
        page = requests.get(f"https://www.profital.ch/de/fl/81450-migros-filialen?page={j}", headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")
        stores = soup.findAll("div", {"class": "wrapper-store-top"})
        
        for i in range(len(stores)):
            try:
                address = stores[i].find("address").text.strip().split("\n")[0].strip() + " "+stores[i].find("address").text.strip().split("\n")[1].strip()
            except AttributeError:
                address = np.nan
            try:
                name = stores[i].find("span", {"class": "title-store"}).text
            except AttributeError:
                name = np.nan
    
            store_dict = {
                'name': name,
                'address': address
            }
    
            df = pd.concat([df, pd.DataFrame(store_dict, index=[0])], ignore_index=True)
    return df

In [94]:
migros_branches = get_filialen()

In [95]:
migros_branches

,name,address
0,Migros,Wengistrasse 7 8004 Zürich
1,Migros,Sihlpassage 8004 Zürich
2,Migros,Zollstrasse 6 8005 Zürich
3,Migros,Löwenstrasse 31 - 35 8001 Zürich
4,Migros,Limmatstrasse 152 8005 Zürich
...,...,...
635,Migros,Place du Lignon 29 a 1219 Le Lignon
636,Migros,Rue des Bossons 78 - 80 1213 Petit-Lancy
637,Migros,Av. des Communes-Réunies 52 1212 Grand-Lancy
638,Migros,Route de Saint-Julien 176 1228 Plan-les-Ouates


In [96]:
migros_branches.to_csv("../data/Migros_branches/migros_branches.csv")